In [1]:
import yfinance as yf
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots


# 30m 1h 1d

In [65]:
ticker = "AAPL"

# Download 30-minute, 1-hour, and 1-day data for the past 60 days
thirty_min_data = yf.download(ticker, period="60d", interval="30m")
hourly_data = yf.download(ticker, period="60d", interval="1h")
daily_data = yf.download(ticker, period="60d", interval="1d")

# Reset index to ensure "Date" column is available
thirty_min_data.reset_index(inplace=True)
hourly_data.reset_index(inplace=True)
daily_data.reset_index(inplace=True)

daily_data
# daily_data.rename(columns={'Date': 'Datetime'}, inplace=True)

daily_data['Datetime'] = daily_data['Date'] + pd.to_timedelta('16:00:00')


c:\Users\edton\miniconda3\envs\trading\lib\site-packages\yfinance\scrapers\history.py:231: FutureWarning:

'T' is deprecated and will be removed in a future version, please use 'min' instead.

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [56]:
thirty_min_data.head(5)

,Datetime,Open,High,Low,Close,Adj Close,Volume
0,2024-07-12 09:30:00,229.000000,231.389999,228.679993,231.350006,231.350006,9178818
1,2024-07-12 10:00:00,231.350006,231.500000,230.300003,230.880005,230.880005,4901328
2,2024-07-12 10:30:00,230.880005,232.149994,230.619995,231.850006,231.850006,3877854
3,2024-07-12 11:00:00,231.839996,232.397995,231.389999,231.899902,231.899902,3649894
4,2024-07-12 11:30:00,231.899994,231.990005,230.839996,230.900101,230.900101,2750432


In [57]:
hourly_data.head(5)

,Datetime,Open,High,Low,Close,Adj Close,Volume
0,2024-07-12 09:30:00-04:00,229.000000,231.500000,228.679993,230.880005,230.880005,14080146
1,2024-07-12 10:30:00-04:00,230.880005,232.397995,230.619995,231.899902,231.899902,7527748
2,2024-07-12 11:30:00-04:00,231.899994,231.990005,230.509995,230.710007,230.710007,5014273
3,2024-07-12 12:30:00-04:00,230.710007,231.910004,230.570007,231.902496,231.902496,4220317
4,2024-07-12 13:30:00-04:00,231.899994,232.639999,231.839996,232.270004,232.270004,4525973


In [58]:
daily_data.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume,Datetime
0,2024-07-12,228.919998,232.639999,228.679993,230.539993,230.273453,53046500,2024-07-12 12:00:00
1,2024-07-15,236.479996,237.229996,233.089996,234.399994,234.128998,62631300,2024-07-15 12:00:00
2,2024-07-16,235.000000,236.270004,232.330002,234.820007,234.548523,43234300,2024-07-16 12:00:00
3,2024-07-17,229.449997,231.460007,226.639999,228.880005,228.615387,57345900,2024-07-17 12:00:00
4,2024-07-18,230.279999,230.440002,222.270004,224.179993,223.920807,66034600,2024-07-18 12:00:00


In [75]:
# Create candlestick traces for each time frame
thirty_min_trace = go.Candlestick(x=thirty_min_data["Datetime"],
                                  open=thirty_min_data["Open"],
                                  high=thirty_min_data["High"],
                                  low=thirty_min_data["Low"],
                                  close=thirty_min_data["Close"],
                                  name="30-Min",
                                  increasing_line_color="green", decreasing_line_color="red")

hourly_trace = go.Candlestick(x=hourly_data["Datetime"],
                              open=hourly_data["Open"],
                              high=hourly_data["High"],
                              low=hourly_data["Low"],
                              close=hourly_data["Close"],
                              name="1-Hour",
                              increasing_line_color="green", decreasing_line_color="red")

daily_trace = go.Candlestick(x=daily_data["Datetime"],
                             open=daily_data["Open"],
                             high=daily_data["High"],
                             low=daily_data["Low"],
                             close=daily_data["Close"],
                             name="1-Day",
                             increasing_line_color="green", decreasing_line_color="red")

# Create a figure and add all three traces, but initially hide them
fig = go.Figure()

fig.add_trace(daily_trace)
fig.add_trace(hourly_trace)
fig.add_trace(thirty_min_trace)


for dat in [daily_data, hourly_data, thirty_min_data]:
    ma_trace = go.Scatter(
                    x=dat["Datetime"],
                    y=dat["Close"].rolling(window=10).mean(),
                    line=dict(width=0.8),
                    # name="ma",
                )
    
    fig.add_trace(ma_trace)


# # Initially hide all but the 1-day trace
# fig.update_traces(visible=False)
# fig.data[0].visible = True  # Show 1-day data by default

# Add buttons to toggle between different charts
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            x=1,
            xanchor="right",
            y=1.2,
            buttons=[
                dict(
                    label="1-Day",
                    method="update",
                    args=[{"visible": [True, False, False]},
                          {"title": f"{ticker} 1-Day Candlestick Chart"}]
                ),
                dict(
                    label="1-Hour",
                    method="update",
                    args=[{"visible": [False, True, False]},
                          {"title": f"{ticker} 1-hour Candlestick Chart"}]
                ),
                dict(
                    label="30-Minute",
                    method="update",
                    args=[{"visible": [False, False, True]},
                          {"title": f"{ticker} 30-min Candlestick Chart"}]
                ),
            ],
            pad={"r": 10, "t": 10},
            showactive=True,
        )
    ]
)

# Customize the layout
fig.update_layout(
    xaxis=dict(rangebreaks=[dict(bounds=["sat", "mon"]), 
                            dict(bounds=[19, 6], pattern="hour")]),
    title=f"{ticker} Candlestick Chart",
    xaxis_title="Date",
    yaxis_title="Price",
    height=600,
    xaxis_rangeslider_visible=False,  # Disable the range slider for cleaner look
    hovermode="x unified",
)

# Show the figure
fig.show()


# 1h 1d

In [133]:
ticker = "meta"

# Download 30-minute, 1-hour, and 1-day data for the past 60 days
hourly_data = yf.download(ticker, period="400d", interval="1h")
daily_data = yf.download(ticker, period="400d", interval="1d")

# Reset index to ensure "Date" column is available
# hourly_data.reset_index(inplace=True)
# daily_data.reset_index(inplace=True)

# daily_data['Datetime'] = daily_data['Date'] + pd.to_timedelta('16:00:00')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [134]:
data = daily_data

fig = make_subplots(
            rows=3,
            cols=1,
            shared_xaxes=True,
            # subplot_titles=("Candle Chart", "Volume", "PSAR"),
            vertical_spacing=0.01,
            row_width=[0.2, 0.2, 0.7],
        )

# Calculate GMMA
# Short-term EMAs (3, 5, 8, 10, 12, 15)
short_term_periods = [3, 5, 8, 10, 12, 15]
for period in short_term_periods:
    data[f'Short EMA {period}'] = data['Close'].ewm(span=period, adjust=False).mean()

# Long-term EMAs (30, 35, 40, 45, 50, 60)
long_term_periods = [30, 35, 40, 45, 50, 60]
for period in long_term_periods:
    data[f'Long EMA {period}'] = data['Close'].ewm(span=period, adjust=False).mean()

# Create traces for the plot
candle_trace = go.Candlestick(
    x=data.index,
    open=data['Open'],
    high=data['High'],
    low=data['Low'],
    close=data['Close'],
    name='Candlestick',
    increasing_line_color='green',
    decreasing_line_color='red',
)

fig.add_trace(
        candle_trace,
        row=1, col=1
    )

# Add GMMA traces
for period in short_term_periods:
    fig.add_trace(
        go.Scatter(x=data.index, y=data[f'Short EMA {period}'], mode='lines', name=f'Short EMA {period}', line=dict(width=1.5)),
        row=1, col=1
    )

for period in long_term_periods:
    fig.add_trace(
        go.Scatter(x=data.index, y=data[f'Long EMA {period}'], mode='lines', name=f'Long EMA {period}', line=dict(width=1.5, dash='dash')),
        row=1, col=1
    )


data['short mom'] = data[f'Short EMA 3'] - data[f'Short EMA 15']
data['long mom'] = data[f'Long EMA 30'] - data[f'Long EMA 60']
data['long short'] = data[f'Short EMA 8'] - data[f'Long EMA 45']

fig.add_trace(go.Bar(x=data.index, y=data[f'short mom'], name=f'short mom'),
              row=2, col=1)
fig.add_trace(go.Bar(x=data.index, y=data[f'long mom'], name=f'long mom'),
              row=2, col=1)
fig.add_trace(go.Bar(x=data.index, y=data[f'long short'], name=f'long short'),
              row=2, col=1)


fig.add_trace(go.Bar(x=data.index, y=data[f'short mom'].pct_change(), name=f'short mom'),
              row=3, col=1)
fig.add_trace(go.Bar(x=data.index, y=data[f'long mom'].pct_change(), name=f'long mom'),
              row=3, col=1)
fig.add_trace(go.Bar(x=data.index, y=data[f'long short'].pct_change(), name=f'long short'),
              row=3, col=1)


# Update layout
fig.update_layout(
    title=f'{ticker} GMMA Chart',
    height=1000,
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False,
    hovermode="x unified",
)

# Show the figure
fig.show()


In [127]:
data['chg rate'] = data['Close'].pct_change().rolling(5).mean() * 100
data['daily fluc'] = (data['High'] - data['Low']).rolling(5).mean() * 0.1

short = go.Bar(x=data.index, y=data[f'chg rate'], name=f'chg rate')
long = go.Bar(x=data.index, y=data[f'daily fluc'], name=f'daily fluc')

fig = go.Figure()
fig.add_trace(short)
fig.add_trace(long)
fig.show()